# A solution for NSUCRYPTO 2020
- Problem 10: _AES-GCM_
- By: _ndh_

**_Answer for Q1_**:

The most common mistake is nonce/IV reuse. Let's check for it:

In [1]:
from collections import namedtuple

Ciphertext = namedtuple("Ciphertext", ("iv", "payload"))
ciphertexts = []
for i in range(8):
    with open(f"AES-GCM-Task_1/{i}.message", "rb") as f:
        content = f.read()
    ciphertexts.append(Ciphertext(
        iv=content[8:20], payload=content[20:-16]))
    
for i in range(8):
    print(i, len(ciphertexts[i].payload), ciphertexts[i].iv)

0 29 b'\x96>\xa6\x16\xe2\xae\xaf>\x9bg^\xcf'
1 18 b'\xeb\x92U\x007\x17\xc7\xdc\xea\x8d\xd8\x05'
2 25 b'\xc5\xe8\x0e\xe2\x15\x1b\xaf\xaf\xda\x16GZ'
3 17 b'\x14anl\xed\xda\x18\xbe\xef\xcc\xdd\x82'
4 27 b'\xfa\x1a\xdft\xf5\xf4\x8c\x01\x88x!\xe3'
5 20 b'\x96>\xa6\x16\xe2\xae\xaf>\x9bg^\xcf'
6 46 b'\x96>\xa6\x16\xe2\xae\xaf>\x9bg^\xcf'
7 20 b'\xc5\xe8\x0e\xe2\x15\x1b\xaf\xaf\xda\x16GZ'


As we can see, the messages at indices 0, 5 and 6 have been encrypted using the same IV. Since the corresponding plaintext for message 0 is known, we can XOR it with the encrypted payload to obtain the keystream, then use the keystream to decrypt upto first 29 bytes (the length of the first message) of the other two messages:

In [2]:
def xor(s1, s2):
    return bytes(c1 ^ c2 for c1, c2 in zip(s1, s2))

keystream = xor(b"Hello, Bob! How's everything?", ciphertexts[0].payload)
print(xor(keystream, ciphertexts[5].payload))
print(xor(keystream, ciphertexts[6].payload))

b'Lincoln Park, 10:15.'
b'Nostalgia is a eternal motif '


**_Answer for Q2_**:

Again, let's check for nonce/IV reuse:

In [3]:
Ciphertext_v2 = namedtuple("Ciphertext", ("iv", "aad", "payload", "tag"))
ciphertexts = []
for i in range(8):
    with open(f"AES-GCM-Task_2/{i}.message", "rb") as f:
        content = f.read()
    ciphertexts.append(Ciphertext_v2(
        iv=content[8:20], aad=content[:20],
        payload=content[20:-16], tag=content[-16:]))

for i in range(8):
    print("%d %3d %s" % (i, len(ciphertexts[i].payload), ciphertexts[i].iv))

0 116 b'\x8f$\x12t?\x9bnst\xa5\xa0\x13'
1 260 b'J\xecw\xd7\x18E6R\x0eN K'
2 106 b'>\xd1\xeb<\xf9\xf3,)\xcc\xbff\xcc'
3  35 b'\x83\x0f\xaa\x00\x91Q\xef\x1fP}\x1f\xf0'
4  92 b'\xa5\xc0\xb1,t\xe9QL7*`}'
5  26 b'b3\xde!v\xdeOU(&*\xfc'
6  71 b'J\xecw\xd7\x18E6R\x0eN K'
7  61 b'\xdb\x03\xb0\rk\xedo\xa1\x8f\x12\x14L'


As we can see, the IV for message 1 and message 6 are the same. So, we have a chance to recover the key-dependent value $H$ and $E_K(\text{Counter 0})$ corresponding to that IV, and therefore, be able to forge the authentication tag for arbitrary ciphertext and additional authenticated data of our choice (for that IV). Let's implement the attack.

Firstly, let's define some utilities that convert back and forth between a block of data and an element of $\mathbb{F}_{128}$:

In [4]:
from sage.all import GF  # https://www.sagemath.org/
from bitarray import bitarray  # https://pypi.org/project/bitarray/

# the finite field of 2**128 elements
F128 = GF(2**128, name='t', modulus=[1, 1, 1, 0, 0, 0, 0, 1] + [0] * 120 + [1])


def block_to_fe(b):
    """Convert a block (<= 16 byte) to an element of F128."""
    ba = bitarray()
    ba.frombytes(b)
    return F128(list(ba))


def fe_to_block(e):
    """Convert an element of F128 to a 16-byte block."""
    b = bitarray(list(e.polynomial())).tobytes()
    b = b.ljust(16, b'\x00')
    return b

Here's the most important function needed for the attack:

In [5]:
from sage.all import PolynomialRing

# the polynomial ring over F128
PR = PolynomialRing(F128, names="x, y")


def get_poly(aad, ct, tag):
    """Given some additional authenticated data `aad`, a ciphertext `ct` and
    a valid authentication tag `tag`, return a polynomial f(X, Y) over F128 
    such that f(H, E(Counter_0)) = 0."""
    coeffs = [block_to_fe(aad[i: i + 16]) for i in range(0, len(aad), 16)]
    coeffs += [block_to_fe(ct[i: i + 16]) for i in range(0, len(ct), 16)]
    coeffs.append(block_to_fe(
        (len(aad) * 8 * 2**64 + len(ct) * 8).to_bytes(16, "big")))

    x, y = PR.gens()
    f = sum(c * x**i for i, c in enumerate(coeffs[::-1]))
    return x * f + y - block_to_fe(tag)

Let's obtain the polynomials for message 1 and message 6:

In [6]:
f1 = get_poly(ciphertexts[1].aad, ciphertexts[1].payload, ciphertexts[1].tag)
f6 = get_poly(ciphertexts[6].aad, ciphertexts[6].payload, ciphertexts[6].tag)

Subtracting one polynomial from another will eliminate `y`, then taking the roots of the result polynomial will give us a list of possible values for $H$:


In [7]:
H_cands = (f1 - f6).univariate_polynomial().roots()
print(len(H_cands))

1


Luckily, we only have one candidate for $H$, so $H$ must be that value. With $H$, computing $E_K(\text{Counter 0})$ is straight-forward:

In [8]:
H, _ = H_cands[0]
EC0, _ = f1.subs(x=H).univariate_polynomial().roots()[0]

Now, we can forge a valid authentication tag for arbitrary `aad` and `ct` of our choice, as long as the IV stays the same. In our situation, we can modify the 8-byte header and change the encrypted payload of message 1 (or 6). For example:

In [9]:
iv = ciphertexts[1].iv
new_payload = b" Hey! Don't use the same IV twice! "
new_header = b"\x00" * 8
new_aad = new_header + iv
f = get_poly(new_aad, new_payload, b'')
new_tag = fe_to_block(F128(f.subs(x=H, y=EC0)))
new_ciphertext = new_header + iv + new_payload + new_tag
print(new_ciphertext)

b"\x00\x00\x00\x00\x00\x00\x00\x00J\xecw\xd7\x18E6R\x0eN K Hey! Don't use the same IV twice! \xa6:\xb6\x94v\xe7\xc5\x1e\x7f0\x17\x81\xfa\xe1\xe4p"


However, the plaintext corresponding to the new payload is not under our control.

**_Answer for Q3_**:

What the new scheme does is that it introduces a new indeterminate to our polynomials: the unknown last 8 bytes of `aad` named $X$. So, we just need more cases of IV reuse to recover them all. Again, let's check for IV reuse:

In [10]:
Ciphertext_v3 = namedtuple("Ciphertext", ("iv", "partial_aad", "payload", "tag"))
ciphertexts = []
for i in range(8):
    with open(f"AES-GCM-Task_3/{i}.message", "rb") as f:
        content = f.read()
    ciphertexts.append(Ciphertext_v3(
        iv=content[8:20], partial_aad=content[:20],
        payload=content[20:-16], tag=content[-16:]))

for i in range(8):
    print(i, len(ciphertexts[i].payload), ciphertexts[i].iv)

0 438 b'\x97\x80/\xbd|\xf3K\xe1\r\x8e\x91\x19'
1 320 b'\xfeM\xeb\x8b\x8e\x82\x12\xa0\x0b\xdd\xd1\xec'
2 431 b'\xc9\x11\xe5\xdd!\x89q\xd6Y\xb0\x1a\xc3'
3 320 b'\xfeM\xeb\x8b\x8e\x82\x12\xa0\x0b\xdd\xd1\xec'
4 108 b'\xc9\x11\xe5\xdd!\x89q\xd6Y\xb0\x1a\xc3'
5 207 b'\x85\xe4\xcd\xd7\xf7\xd2\xd2\xadi\xb5q\xc1'
6 970 b'>\xe3\x11\x94\xe8 \xb4\x11=\xe9\x9e}'
7 320 b'\xfeM\xeb\x8b\x8e\x82\x12\xa0\x0b\xdd\xd1\xec'


As we can see, messages 1, 3, and 7 have the same IV. So do messages 2, 4. Let's make a new version for `get_poly`:

In [11]:
# add a new indeterminate `z` which represents X
PR_v2 = PolynomialRing(F128, names="x, y, z")

def get_poly_v2(partial_aad, n, ct, tag):
    """Given some partial additional authenticated data `partial_aad`, the size
    `n` of the unknown additional authenticated data ending part X, a 
    ciphertext `ct` and a valid authentication tag `tag`, return a polynomial
    f(x, y, z) over F128 such that f(H, E(Counter_0), X) = 0."""
    # only support X inside 1 block
    r = len(partial_aad) % 16
    assert n > 0 and r + n <= 16
    
    t = F128.gen()
    x, y, z = PR_v2.gens()
    coeffs = [block_to_fe(partial_aad[i: i + 16]) 
              for i in range(0, len(partial_aad), 16)]
    if r == 0:  # X lying at the beginning of a new block
        coeffs.append(z)
    else:  # X lying in the last block
        coeffs[-1] += t**(8 * r)*z
    coeffs += [block_to_fe(ct[i: i + 16]) for i in range(0, len(ct), 16)]
    coeffs.append(block_to_fe(
        ((len(partial_aad) + n) * 8 * 2**64 + len(ct) * 8).to_bytes(16, "big")
    ))

    f = sum(c * x**i for i, c in enumerate(coeffs[::-1]))
    return x * f + y - block_to_fe(tag)

Now, let's obtain the polynomials for the involved messages:

In [12]:
f1 = get_poly_v2(ciphertexts[1].partial_aad, 8, ciphertexts[1].payload, ciphertexts[1].tag)
f3 = get_poly_v2(ciphertexts[3].partial_aad, 8, ciphertexts[3].payload, ciphertexts[3].tag)
f7 = get_poly_v2(ciphertexts[7].partial_aad, 8, ciphertexts[7].payload, ciphertexts[7].tag)

f2 = get_poly_v2(ciphertexts[2].partial_aad, 8, ciphertexts[2].payload, ciphertexts[2].tag)
f4 = get_poly_v2(ciphertexts[4].partial_aad, 8, ciphertexts[4].payload, ciphertexts[4].tag)

Since messages 1, 3 and 7 have the same payload size, subtract one polynomial in the set $\{f_1, f_3, f_7\}$ from another will eliminate both `y` and `z` at the same time. Hence, we have 2 polynomials having the same root `x = H`. As a result, $H$ is likely to be uniquely determined (without luck) this time:

In [13]:
H_cands = (f1 - f3).gcd(f1 - f7).univariate_polynomial().roots()
assert len(H_cands) == 1
H, _ = H_cands[0]

From now on, we can modify the headers and the encrypted payloads of messages 1, 3, and 7, but the payload size (in term of 16-byte blocks) can not be changed! To change the size to arbitrary value we want, $X$ and $E_K(\text{Counter 0})$ must be also recovered.

Since the payloads of messages 2, 4 are quite different in size (431 vs 108), the two indeterminates `y` and `z` can be separated in this case. By reusing the value $H$ found above (note that $E_K(\text{Counter 0})$ depends on both the key and the IV, while $H$ and $X$ only depends on the key), we are able to recover $X$ and $E_K(\text{Counter 0})$:

In [14]:
X, _ = (f2 - f4).subs(x=H).univariate_polynomial().roots()[0]
print(X)

t^63 + t^62 + t^61 + t^60 + t^58 + t^57 + t^55 + t^49 + t^47 + t^45 + t^44 + t^42 + t^41 + t^40 + t^39 + t^38 + t^37 + t^34 + t^32 + t^30 + t^29 + t^26 + t^25 + t^20 + t^18 + t^17 + t^16 + t^15 + t^13 + t^11 + t^9 + t^8 + t^7 + t^5 + t^4 + t^2 + t + 1


As we can see, $X$ as polynomial over $\mathbb{F}_2$ only has a degree of 63. This is expected since the size of $X$ as bytes is only 8. Now, let's recover the $E_K(\text{Counter 0})$ for both the IV shared by messages 1, 3, 7 and the one shared by messages 2, 4:

In [15]:
EC0_137, _ = f1.subs(x=H, z=X).univariate_polynomial().roots()[0]
EC0_24, _ = f2.subs(x=H, z=X).univariate_polynomial().roots()[0]

Now, we are able to forge a valid tag for arbitrary `aad` and `ct` of our choice, as long as `iv` be one of the two IVs that have been reused. For example:

In [16]:
iv_137 = ciphertexts[1].iv
new_payload = b" Hey! Don't use the same IV twice! "
new_header = b"\x00" * 8
new_partial_aad = new_header + iv_137
f = get_poly_v2(new_partial_aad, 8, new_payload, b'')
new_tag = fe_to_block(F128(f.subs(x=H, y=EC0_137, z=X)))
new_ciphertext = new_header + iv_137 + new_payload + new_tag
print(new_ciphertext)

b"\x00\x00\x00\x00\x00\x00\x00\x00\xfeM\xeb\x8b\x8e\x82\x12\xa0\x0b\xdd\xd1\xec Hey! Don't use the same IV twice! \xc4\xab|NI\x19\xe5\xcc\x8fTo@&\xa3\xe9V"
